### Inspect Subtraction in Detail
Michael Wood-Vasey

2019-06-27

Some starting points to inspect DIA Processing

After working through this Notebook, a person should be able to
1. Load a set of coadd, science, difference image for a given visit, filter, tract+path.
2. Load a postage stamps of coadd, science, and difference images
3. Load the PSF object for an image and inspect the size of the PSF.
4. Subtract two PSFs from different images

Future Goals of this Notebook:
5. Understand PSF of each image and how the Image Difference software calculated the convolution between them.
6. Inspect convolution kernels in both ZOGY and A&L.

In [ ]:
import os
import sys

import numpy as np

import lsst.afw.display as afwDisplay
import lsst.afw.geom as afwGeom
from lsst.daf.persistence import Butler
from lsst.geom import SpherePoint

In [ ]:
# The Postage Stamp routines are saved in a separate file here in the same folder as this Notebook
from image_stamp import make_cutout_image

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

Inspecting a set of subtractions provided by Bob Armstrong:
#desc-dc2-dia

2019-05-03

Bob:
"
I have processed a new batch of difference images for Run 1.2p.  This new batch creates templates from the  first two years that only uses visits with seeing < 0.7.  It then produces difference images for visits from the remaining years.  It combines all the filters except for u-band where there were not enough visits in the first two years.  This new run also produces forced photometry on the templates at the diaObject positions.  It uses the same default settings of Alard-Lupton for difference image as the run before.
"

In [ ]:
repo = '/global/cscratch1/sd/rearmstr/new_templates/diffim_template'

------

In [ ]:
butler = Butler(repo)

Here is the information necessary to find our sample images.  These are called "Data IDs" in the DM Science Pipeline language.

The template is fixed for the given repo.  Then we pick a good subtraction and a subtraction with clear ringing.

In [ ]:
tract, patch, filt = 4849, '6,6', 'r'
template_id = {'tract': tract, 'patch': patch, 'filter': filt}
good_id = {'visit': 1181556, 'raftName': 'R12', 'detector': 45, 'filter': filt}
ring_id = {'visit': 1203190, 'detector': 38, 'filter': filt}

Note that the calexps are from the central Run 1.2p processing, which is a parent of the repo used for these difference image tests.

In [ ]:
tmpl = butler.get(datasetType='deepCoadd', dataId=template_id)

new_good = butler.get(datasetType='calexp', dataId=good_id)
sub_good = butler.get(datasetType='deepDiff_differenceExp', dataId=good_id)

new_ring = butler.get(datasetType='calexp', dataId=ring_id)
sub_ring = butler.get(datasetType='deepDiff_differenceExp', dataId=ring_id)

In [ ]:
# Reading the diaSrc catalogs needs an updated DM Science Pipelines version to work.  
# src_good = butler.get(datasetType='deepDiff_diaSrc', dataId=good_id)
# src_ring = butler.get(datasetType='deepDiff_diaSrc', dataId=ring_id)

# The `src_good` and `src_ring` are the AFW Tables from the catalogs
# You might be more familar with them as AstroPy tables.
# src_good = src_good.asAstropy()
# src_ring = src_ring.asAstropy()

In [ ]:
sample_star = {'ra': 53.069214, 'dec': -28.343584}
sample_agn = {'ra': 53.135801, 'dec': -28.426165, 'diaObjectId': 21326977935867913}

In [ ]:
ra, dec = sample_agn['ra'], sample_agn['dec']

In [ ]:
title = 'Template: AGN'
agn_tmpl_cutout = make_cutout_image(butler, template_id, ra, dec, dataset_type='deepCoadd', title=title)

In [ ]:
title = 'Good Subtraction: AGN'
agn_good_cutout = make_cutout_image(butler, good_id, ra, dec, dataset_type='deepDiff_differenceExp', title=title)

In [ ]:
title = 'Ringing Subtraction: AGN'
agn_ring_cutout = make_cutout_image(butler, ring_id, ra, dec, dataset_type='deepDiff_differenceExp', title=title)

In [ ]:
ra, dec = sample_star['ra'], sample_star['dec']

In [ ]:
title = 'Template: Star'
star_tmpl_cutout = make_cutout_image(butler, template_id, ra, dec, dataset_type='deepCoadd', title=title)

In [ ]:
title = 'Good Subtraction: Star'
star_good_cutout = make_cutout_image(butler, good_id, ra, dec, dataset_type='deepDiff_differenceExp', title=title)

We can show the subtraction in the reference frame of the template, but note that this resamples the pixels.  I mostly present it here to show how to do it.

In the subtraction process itself, the opposite thing was done.  The template was warped to the science image.

In [ ]:
title = 'Good Subtraction: Star -- warped to template frame'
star_good_cutout = make_cutout_image(butler, good_id, ra, dec, dataset_type='deepDiff_differenceExp', title=title,
                                     warp_to_exposure=star_tmpl_cutout)

In [ ]:
title = 'Ringing Subtraction: Star'
star_ring_cutout = make_cutout_image(butler, ring_id, ra, dec, dataset_type='deepDiff_differenceExp', title=title)

In [ ]:
title = 'Ringing Subtraction: Star - warped to template frame'
star_ring_cutout = make_cutout_image(butler, ring_id, ra, dec, dataset_type='deepDiff_differenceExp', title=title,
                                     warp_to_exposure=star_tmpl_cutout)

In [ ]:
radec = SpherePoint(ra, dec, afwGeom.degrees)

In [ ]:
new_good_psf = new_good.getPsf()
xy = afwGeom.PointD(new_good.getWcs().skyToPixel(radec))

new_good_quad_shape = new_good_psf.computeShape(xy)
new_good_kernel_image = new_good_psf.computeKernelImage(xy)

print(new_good_quad_shape)

In [ ]:
display = afwDisplay.Display(backend='matplotlib')
display.mtv(new_good_kernel_image)
display.show_colorbar()

In [ ]:
new_ring_psf = new_ring.getPsf()

xy = afwGeom.PointD(new_ring.getWcs().skyToPixel(radec))

new_ring_quad_shape = new_ring_psf.computeShape(xy)
new_ring_kernel_image = new_ring_psf.computeKernelImage(xy)

print(new_ring_quad_shape)

In [ ]:
display = afwDisplay.Display(backend='matplotlib')
display.mtv(new_ring_kernel_image)
display.show_colorbar()

In [ ]:
tmpl_psf = tmpl.getPsf()

xy = afwGeom.PointD(tmpl.getWcs().skyToPixel(radec))

tmpl_quad_shape = tmpl_psf.computeShape(xy)
tmpl_kernel_image = tmpl_psf.computeKernelImage(xy)

print(tmpl_quad_shape)

In [ ]:
display = afwDisplay.Display(backend='matplotlib')
display.mtv(tmpl_kernel_image)
display.show_colorbar()

In [ ]:
new_good_len = len(new_good_kernel_image.array)
tmpl_len = len(tmpl_kernel_image.array)
min_len = min(new_good_len, tmpl_len)

def get_center_box(im, size):
    nx, ny = im.array.shape
    bbox = afwGeom.Box2I(minimum=afwGeom.Point2I(x=-size//2+1, y=-size//2+1),
                         maximum=afwGeom.Point2I(x=+size//2, y=+size//2))
    return im[bbox]

new_good_center = get_center_box(new_good_kernel_image, min_len)
tmpl_center = get_center_box(tmpl_kernel_image, min_len)

# Renorm
new_good_center /= np.sum(new_good_center.array)
tmpl_center /= np.sum(tmpl_center.array)

from copy import copy, deepcopy
diff = deepcopy(new_good_center)
diff -= tmpl_center

In [ ]:
display = afwDisplay.Display(backend='matplotlib')
display.mtv(diff)
display.show_colorbar()

In [ ]:
# Check normalizations
print(np.sum(tmpl_center.array))
print(np.sum(new_good_center.array))
print(np.sum(diff.array))

### Appendix
#### A. Just Get Me the Files!

While the following feature is not meant to be supported in the long-term (the Butler may eventually provide access to datasets across filesystems, remote, cloud buckets, etc.), for debugging and visualization it remains useful to get the direct filename.  We can get that using the Butler `getUri` function ("get Uniform Resource Identifier").

In [ ]:
template_file = butler.getUri(datasetType='deepCoadd', dataId=template_id)
good_new_file = butler.getUri(datasetType='calexp', dataId=good_id)
ring_new_file = butler.getUri(datasetType='calexp', dataId=ring_id)
good_sub_file = butler.getUri(datasetType='deepDiff_differenceExp', dataId=good_id)
ring_sub_file = butler.getUri(datasetType='deepDiff_differenceExp', dataId=ring_id)

If, for example you wanted to download these files to you local machine, you could do the following:
    
This isn't the recommended approach and doesn't scale to lots of subtractions, but I (MWV) found it useful when investigating things are our very early stages in understanding DC2 DIA.

In [ ]:
machine = 'cori.nersc.gov'
files_to_copy = [template_file, good_new_file, ring_new_file, good_sub_file, ring_sub_file]

for f in files_to_copy:
    print(f'rsync {machine}:{f} ./')